# Home Credit Prediction: Data Cleaning of POS CASH Table


For an up-to-date version / full view of the plotly - plots, please, go to

Data Cleaning - POS CASH:  https://drive.google.com/file/d/16n5gXaxqB59kyMqAB5thSLDFwrxoqg0u/view?usp=sharing

List of all notebooks and resources for this project: https://drive.google.com/file/d/1Z8vPNZAcivWOxeh3UKFfeARbQCMkQ_NR/view?usp=sharing

## Import Modules

In [3]:
%%capture
#! pip install -q pingouin
#! pip install -q scikit-optimize
! pip install -q scikit-optimize

In [4]:
import numpy as np
import pandas as pd

import sys
import os
import warnings
from importlib import reload

from dask import dataframe as dd
#import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px

from google.colab import drive
drive.mount("/content/gdrive")

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
#pd.reset_option('display.max_rows')

Mounted at /content/gdrive


In [5]:
pd.__version__

'2.1.4'

In [6]:
home_folder = '/content/gdrive/MyDrive/Colab Notebooks/Portfolio/ML_HomeCredit_DefaultRiskEvaluation/'

### Functions

The Python-file with the functions is at
https://drive.google.com/file/d/17IchsTGy2QI9sq0LTIvGvxAk2mrWs4Xz/view?usp=sharing

In [7]:
%load_ext autoreload
%autoreload 2

sys.path.append(home_folder)
import driskfunc as dfunc

# 1. Load and Update Data

data source: https://storage.googleapis.com/341-home-credit-default/home-credit-default-risk.zip

description: https://storage.googleapis.com/341-home-credit-default/Home%20Credit%20Default%20Risk.pdf

In [8]:
HCdescr = pd.read_csv(home_folder+'data/HomeCredit_columns_description.csv', encoding='latin1') #, dtype=dtype)


In [9]:
HCdescr.loc[HCdescr.Table == 'POS_CASH_balance.csv']

,Unnamed: 0,Table,Row,Description,Special
142,145,POS_CASH_balance.csv,SK_ID_PREV,"ID of previous credit in Home Credit related to loan in our sample. (One loan in our sample can have 0,1,2 or more previous loans in Home Credit)",NaN
143,146,POS_CASH_balance.csv,SK_ID_CURR,ID of loan in our sample,NaN
144,147,POS_CASH_balance.csv,MONTHS_BALANCE,"Month of balance relative to application date (-1 means the information to the freshest monthly snapshot, 0 means the information at application - often it will be the same as -1 as many banks are not updating the information to Credit Bureau regularly )",time only relative to the application
145,148,POS_CASH_balance.csv,CNT_INSTALMENT,Term of previous credit (can change over time),NaN
146,149,POS_CASH_balance.csv,CNT_INSTALMENT_FUTURE,Installments left to pay on the previous credit,NaN
147,150,POS_CASH_balance.csv,NAME_CONTRACT_STATUS,Contract status during the month,NaN
148,151,POS_CASH_balance.csv,SK_DPD,DPD (days past due) during the month of previous credit,NaN
149,152,POS_CASH_balance.csv,SK_DPD_DEF,DPD during the month with tolerance (debts with low loan amounts are ignored) of the previous credit,NaN


In [10]:

csv_pc = home_folder+'data/POS_CASH_balance.csv'
HCapp_pc = dd.read_csv(csv_pc) #, dtype=dtype)

df = HCapp_pc
df_name = 'HCapp POS CASH balance'

In [11]:
df.npartitions

6

In [12]:
df = df.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'], ascending=False)
df = df.sort_values(by=['SK_ID_CURR','SK_ID_PREV'], ascending=True)

In [13]:
df.head(100)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
532573,1369693,100001,-53,4.0,0.0,Completed,0,0
1373359,1369693,100001,-54,4.0,1.0,Active,0,0
1156999,1369693,100001,-55,4.0,2.0,Active,0,0
454770,1369693,100001,-56,4.0,3.0,Active,0,0
500325,1369693,100001,-57,4.0,4.0,Active,0,0
1597518,1851984,100001,-93,4.0,0.0,Completed,0,0
197015,1851984,100001,-94,4.0,0.0,Active,0,0
226147,1851984,100001,-95,4.0,1.0,Active,7,7
1261679,1851984,100001,-96,4.0,2.0,Active,0,0
1628483,1038818,100002,-1,24.0,6.0,Active,0,0


In [14]:
size_df = [df.shape[0].compute(),  df.shape[1]]

print('The dataset', df_name, 'has', size_df[0], 'rows and', size_df[1], 'features.')

The dataset HCapp POS CASH balance has 10001358 rows and 8 features.


# 2. Data Cleaning

* Handling missing values.
* Removing duplicate samples and features.
* Remove unneccessary columns/rows.
* Treating (here rather checking) the outliers.

## Check Missing Values and Duplicates

Overview of amounts of Nan and of data type:

In [15]:
dfunc.count_dtypes(df, name = df_name)


The dataset HCapp POS CASH balance has:
5 features of type int64.
2 features of type float64.
1 features of type string.


In [16]:
%%time
%reload_ext autoreload

nan_overview_df = dfunc.nan_type_overview_dd(df, size_df[0])
nan_overview_df.round(1).style.background_gradient(cmap="Blues")

CPU times: user 35.7 s, sys: 5.16 s, total: 40.9 s
Wall time: 27.6 s


,type,NaN[abs],NaN[%]
SK_ID_PREV,int64,0,0.000000
SK_ID_CURR,int64,0,0.000000
MONTHS_BALANCE,int64,0,0.000000
CNT_INSTALMENT,float64,26071,0.300000
CNT_INSTALMENT_FUTURE,float64,26087,0.300000
NAME_CONTRACT_STATUS,string,0,0.000000
SK_DPD,int64,0,0.000000
SK_DPD_DEF,int64,0,0.000000


### Duplicates Check

In [17]:
%reload_ext autoreload

df_dup = dfunc.get_dup_dd(df, name=df_name, size=size_df[0])

Total number of duplicates in " HCapp POS CASH balance " : 0 ( 0.0 %).


## Other Checks and Modifications

#### Overview

In [18]:
df.describe().compute().T.round(1)

,count,mean,std,min,25%,50%,75%,max
SK_ID_PREV,10001358.0,1903216.6,535846.5,1000001.0,1436092.0,1898404.0,2372628.0,2843499.0
SK_ID_CURR,10001358.0,278403.9,102763.7,100001.0,182234.0,273959.0,367081.0,456255.0
MONTHS_BALANCE,10001358.0,-35.0,26.1,-96.0,-54.0,-27.0,-13.0,-1.0
CNT_INSTALMENT,9975287.0,17.1,12.0,1.0,10.0,12.0,24.0,92.0
CNT_INSTALMENT_FUTURE,9975271.0,10.5,11.1,0.0,3.0,7.0,14.0,85.0
SK_DPD,10001358.0,11.6,132.7,0.0,0.0,0.0,0.0,4231.0
SK_DPD_DEF,10001358.0,0.7,32.8,0.0,0.0,0.0,0.0,3595.0


In [19]:
df_obj = df.describe(exclude=np.number).compute().T
df_obj['freq'] = pd.to_numeric(df_obj['freq'])
df_obj['count'] = pd.to_numeric(df_obj['count'])
df_obj['freq[%]'] = (df_obj['freq']/df_obj['count']*100).astype('float')
df_obj.round(1)

,unique,count,top,freq,freq[%]
NAME_CONTRACT_STATUS,9,10001358,Active,9151119,91.5


#### INSTALMENTS paid

In [20]:
df['INSTALMENTS_paid'] = df['CNT_INSTALMENT'] - df['CNT_INSTALMENT_FUTURE'] # align with credit card table

#### NAME_CONTRACT_STATUS

In [21]:
df.NAME_CONTRACT_STATUS.value_counts().compute()

,count
NAME_CONTRACT_STATUS,
Active,9151119
Completed,744883
Signed,87260
Demand,7065
Returned to the store,5461
Approved,4917
Amortized debt,636
Canceled,15
XNA,2


Inspection shows that a month with 'XNA', 'Canceled', 'Returned to the store',  (typicall the very first month of the credit running, i.e. oldest month in payment list of a specific SK_ID_PREV) does not end the contract, but continues with 'Active' in the following months.
Even if 'XNA', 'Canceled', 'Returned to the store' give a hint on the credit worthiness, with just 2, 15, 5461 rows out of >10 Mio. it provides too few cases to show an effect. Maybe 'Refused' rather indicates mistakes when filling the application form.

'Amortized debt' = next escalation level above 'Demand'

636 out of 10Mio rows. ---> add them to 'Demand'



In [22]:
df.loc[df.NAME_CONTRACT_STATUS == 'Returned to the store'].compute().head(5)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,INSTALMENTS_paid
1026728,2299329,100006,-20,NaN,NaN,Returned to the store,0,0,NaN
1453090,1387636,100092,-46,12.0,12.0,Returned to the store,0,0,0.0
1608420,2216247,100094,-8,NaN,NaN,Returned to the store,0,0,NaN
968275,2192263,100101,-42,12.0,11.0,Returned to the store,0,0,1.0
1471089,1752483,100110,-2,NaN,NaN,Returned to the store,0,0,NaN


In [23]:
df.loc[df.SK_ID_PREV == 2299329].compute()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,INSTALMENTS_paid
1260947,2299329,100006,-13,5.0,0.0,Completed,0,0,5.0
1547042,2299329,100006,-14,5.0,0.0,Active,0,0,5.0
1147003,2299329,100006,-15,12.0,8.0,Active,0,0,4.0
1561018,2299329,100006,-16,12.0,9.0,Active,0,0,3.0
1417695,2299329,100006,-17,12.0,10.0,Active,0,0,2.0
1632814,2299329,100006,-18,12.0,11.0,Active,0,0,1.0
419009,2299329,100006,-19,12.0,12.0,Active,0,0,0.0
1026728,2299329,100006,-20,NaN,NaN,Returned to the store,0,0,NaN


In [24]:
df_cstat = df['NAME_CONTRACT_STATUS']
df_cstat = df_cstat.mask((df_cstat=='Signed'), 'Initialized')
df_cstat = df_cstat.mask((df_cstat=='Returned to the store'), 'Initialized')
df_cstat = df_cstat.mask((df_cstat=='Approved'), 'Initialized')
df_cstat = df_cstat.mask((df_cstat=='Canceled'), 'Initialized')
df_cstat = df_cstat.mask((df_cstat=='XNA'), 'Initialized')
df_cstat = df_cstat.mask((df_cstat=='Amortized debt'), 'Demand')

df['NAME_CONTRACT_STATUS_red'] = df_cstat
df['NAME_CONTRACT_STATUS_red'].value_counts().compute()

,count
NAME_CONTRACT_STATUS_red,
Active,9151119
Completed,744883
Initialized,97655
Demand,7701


## Aggregate by SK_ID_PREV

In [25]:
df_stat = df.categorize(columns = ['NAME_CONTRACT_STATUS_red']).NAME_CONTRACT_STATUS_red
df_status_cat = dd.get_dummies(df_stat, prefix = 'STATUS', dtype=int)

df_with_dummies = dd.concat([df, df_status_cat], axis=1)

#df_with_dummies.head(20)

In [26]:
df_mean = df_with_dummies.groupby("SK_ID_PREV").mean(numeric_only=True)
df_mean = df_mean.drop(columns=['MONTHS_BALANCE',
                                'CNT_INSTALMENT',
                                'CNT_INSTALMENT_FUTURE'
                                ])

df_count = df_with_dummies.groupby("SK_ID_PREV")['MONTHS_BALANCE'].count().to_frame()
df_count = df_count.rename(columns={'MONTHS_BALANCE': 'MONTHS_BALANCE_count'})

df_installment = df_with_dummies.groupby("SK_ID_PREV")['MONTHS_BALANCE',
                                                       #CNT_INSTALMENT,
                                                       'CNT_INSTALMENT_FUTURE',
                                                       'NAME_CONTRACT_STATUS',
                                                       'NAME_CONTRACT_STATUS_red'
                                                       ].first()

install_cols = {'MONTHS_BALANCE': 'MONTHS_BALANCE_latest',
                'CNT_INSTALMENT_FUTURE': 'CNT_INSTALMENT_FUTURE_latest',
                'NAME_CONTRACT_STATUS': 'NAME_CONTRACT_STATUS_latest',
                'NAME_CONTRACT_STATUS_red': 'NAME_CONTRACT_STATUS_red_latest'
                }

df_installment = df_installment.rename(columns=install_cols)

df_installment_max = df_with_dummies.groupby("SK_ID_PREV")['CNT_INSTALMENT'
                                                       ].max().to_frame()
df_installment_max = df_installment_max.rename(columns={'CNT_INSTALMENT': 'CNT_INSTALMENT_max'})


df_inst = dd.concat([df_installment_max, df_installment], axis=1)
df_count_inst = dd.concat([df_count, df_inst], axis=1)
df_final = dd.concat([df_mean, df_count_inst], axis=1)

df_final = df_final.reset_index()
df_final=df_final.sort_values(by='SK_ID_CURR')

df_final.round(2).head(20)

,SK_ID_PREV,SK_ID_CURR,SK_DPD,SK_DPD_DEF,INSTALMENTS_paid,STATUS_Active,STATUS_Completed,STATUS_Demand,STATUS_Initialized,MONTHS_BALANCE_count,CNT_INSTALMENT_max,MONTHS_BALANCE_latest,CNT_INSTALMENT_FUTURE_latest,NAME_CONTRACT_STATUS_latest,NAME_CONTRACT_STATUS_red_latest
71457,1851984,100001.0,1.75,1.75,3.25,0.75,0.25,0.0,0.00,4,4.0,-93,0.0,Completed,Completed
31924,1369693,100001.0,0.00,0.00,2.00,0.80,0.20,0.0,0.00,5,4.0,-53,0.0,Completed,Completed
3757,1038818,100002.0,0.00,0.00,9.00,1.00,0.00,0.0,0.00,19,24.0,-1,6.0,Active,Active
67854,1810518,100003.0,0.00,0.00,3.50,0.88,0.12,0.0,0.00,8,12.0,-18,0.0,Completed,Completed
132952,2636178,100003.0,0.00,0.00,3.38,0.88,0.12,0.0,0.00,8,6.0,-21,0.0,Completed,Completed
114374,2396755,100003.0,0.00,0.00,5.50,1.00,0.00,0.0,0.00,12,12.0,-66,1.0,Active,Active
48632,1564014,100004.0,0.00,0.00,1.50,0.75,0.25,0.0,0.00,4,4.0,-24,0.0,Completed,Completed
122020,2495675,100005.0,0.00,0.00,4.50,0.82,0.09,0.0,0.09,11,12.0,-15,0.0,Completed,Completed
98213,2190416,100006.0,0.00,0.00,4.50,1.00,0.00,0.0,0.00,10,12.0,-1,3.0,Active,Active
89362,2078043,100006.0,0.00,0.00,0.67,0.67,0.33,0.0,0.00,3,48.0,-4,0.0,Completed,Completed


In [27]:
df_stat_l = df_final.categorize(columns = ['NAME_CONTRACT_STATUS_red_latest']).NAME_CONTRACT_STATUS_red_latest
df_status_l_cat = dd.get_dummies(df_stat_l, prefix = 'STATUS_latest', dtype=int)

df_with_dummies_final = dd.concat([df_final, df_status_l_cat], axis=1)

#df_with_dummies.head(20)

In [28]:
size_df_final = [df_final.shape[0].compute(),  df_final.shape[1]]

print('The condensed dataset has', size_df_final[0], 'rows and', size_df_final[1], 'features.')
print('Initial size was', size_df[0], 'rows and', size_df[1], 'features.')

The condensed dataset has 936325 rows and 15 features.
Initial size was 10001358 rows and 8 features.


In [29]:
nan_overview_df = dfunc.nan_type_overview_dd(df_final, size_df_final[0])
nan_overview_df.round(1).style.background_gradient(cmap="Blues")

,type,NaN[abs],NaN[%]
SK_ID_PREV,int64,0,0.000000
SK_ID_CURR,float64,0,0.000000
SK_DPD,float64,0,0.000000
SK_DPD_DEF,float64,0,0.000000
INSTALMENTS_paid,float64,890,0.100000
STATUS_Active,float64,0,0.000000
STATUS_Completed,float64,0,0.000000
STATUS_Demand,float64,0,0.000000
STATUS_Initialized,float64,0,0.000000
MONTHS_BALANCE_count,int64,0,0.000000


This modified data set can now be merged with the 'previous applictions' dataset.

# Export

In [30]:
%%capture
! mkdir home_folder+'cleaned/'
df_final.to_csv(home_folder+'cleaned/HC_POS_CASH_cleaned.csv',
                 index=False, single_file = True)